In [1]:
import os, importlib, sys

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

%load_ext autoreload 

In [2]:
# Common paths
BASE_PATH = os.path.join(os.getcwd(), "..", "..")
MODULE_PATH = os.path.join(BASE_PATH, "modules")
DS_PATH = os.path.join(BASE_PATH, "datasets")

In [3]:
sys.path.append(MODULE_PATH)

import mp.MomentPropagation as mp
importlib.reload(mp)

import data.mnist as mnist_loader
importlib.reload(mnist_loader)

<module 'data.mnist' from '/home/exleonem/Desktop/workspace/thesis/wp/notebooks/model/../../modules/data/mnist.py'>

In [4]:
import tensorflow as tf
from tensorflow import keras
import tensorflow.keras
from tensorflow.keras import Sequential
from tensorflow.keras.layers import  Conv2D, MaxPool2D, Dropout, Flatten, Dense, Softmax

In [5]:
gpus = tf.config.experimental.list_physical_devices("GPU")
cpus = tf.config.experimental.list_physical_devices("CPU")

if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
            
        logical_gpus = tf.config.experimental.list_logical_devices("GPU")
        print(len(gpus), "Physical GPU's, ", len(logical_gpus), "Logical GPU's")
        
    except RuntimeError as e:
        print(e)
        
elif cpus:
    try:
        logical_cpus = tf.config.experimental.list_logical_devices("CPU")
        print(len(cpus), "Physical CPU,", len(logical_cpus), "Logical CPU")
        
    except RuntimeError as e:
        print(e)
        
        
tfk = tf.keras

1 Physical GPU's,  1 Logical GPU's


In [6]:
input_shape = (28, 28, 1)
model = Sequential([
    Conv2D(128, 4,activation="relu", input_shape=input_shape),
    MaxPool2D(),
    Dropout(.2),
    Conv2D(64, 3, activation="relu"),
    MaxPool2D(),
    Dropout(.2),
    Flatten(),
    Dense(512, activation="relu"),
    Dense(256, activation="relu"),
    Dense(128, activation="relu"),
    Dense(1, activation="sigmoid")
])

model.build(input_shape)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 25, 25, 128)       2176      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 12, 12, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 12, 12, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 10, 10, 64)        73792     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 5, 5, 64)          0         
_________________________________________________________________
flatten (Flatten)            (None, 1600)              0

In [7]:
%autoreload 2
import active_learning as active
importlib.reload(active)

<module 'active_learning' from '/home/exleonem/Desktop/workspace/thesis/wp/notebooks/model/../../modules/active_learning/__init__.py'>

In [8]:
import mp.MomentPropagation as mp
importlib.reload(mp)

mp = mp.MP()
mp_model = mp.create_MP_Model(model=model, use_mp=True, verbose=True)

In [9]:
import data.mnist as mnist_loader
importlib.reload(mnist_loader)

# Load Data
mnist_path = os.path.join(DS_PATH, "mnist")
inputs, targets = mnist_loader.load(mnist_path)

# Select only first and second class
selector = (targets==0) | (targets==1)
new_inputs = inputs[selector].astype("float32")/255.0
new_targets = targets[selector]

# Create splits
x_train, x_test, y_train, y_test = train_test_split(new_inputs, new_targets)
x_test, x_val, y_test, y_val = train_test_split(x_test, y_test)

In [10]:
new_inputs.shape

(14780, 28, 28)

In [11]:
new_inputs[:, None, ...].shape

(14780, 1, 28, 28)

In [12]:
sample_input = np.random.randn(10, 28, 28, 1)
sample_input.shape

(10, 28, 28, 1)

In [13]:
# pred_mp,var_mp = mp_model(sample_input)

In [14]:
mp_model

In [15]:
%autoreload 2
import bayesian 
from bayesian import McDropout, MomentPropagation

mp_m = MomentPropagation(mp_model)

In [16]:
prediction = mp_m.predict(sample_input)

In [17]:
prediction

[<tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[0.54047096],
        [0.51992303],
        [0.53312427],
        [0.5270455 ],
        [0.5366923 ],
        [0.53868794],
        [0.5346813 ],
        [0.53338724],
        [0.5317819 ],
        [0.5354108 ]], dtype=float32)>,
 <tf.Tensor: shape=(10, 1), dtype=float32, numpy=
 array([[0.0017022 ],
        [0.00170491],
        [0.0016711 ],
        [0.00151935],
        [0.00174694],
        [0.00164654],
        [0.00164448],
        [0.00154968],
        [0.00162749],
        [0.00154184]], dtype=float32)>]

In [18]:
variance = mp_m.variance(prediction)
variance

array([[0.0017022 , 0.9982978 ],
       [0.00170491, 0.99829507],
       [0.0016711 , 0.9983289 ],
       [0.00151935, 0.9984806 ],
       [0.00174694, 0.99825305],
       [0.00164654, 0.9983535 ],
       [0.00164448, 0.9983555 ],
       [0.00154968, 0.99845034],
       [0.00162749, 0.9983725 ],
       [0.00154184, 0.99845815]], dtype=float32)

In [19]:
exepctation = mp_m.expectation(prediction)
exepctation

array([[0.54047096, 0.45952904],
       [0.51992303, 0.48007697],
       [0.53312427, 0.46687573],
       [0.5270455 , 0.4729545 ],
       [0.5366923 , 0.46330768],
       [0.53868794, 0.46131206],
       [0.5346813 , 0.46531868],
       [0.53338724, 0.46661276],
       [0.5317819 , 0.4682181 ],
       [0.5354108 , 0.46458918]], dtype=float32)

In [32]:
%autoreload 2

from acl import ActiveLearning
from active_learning import TrainConfig, Config

import bayesian 
from bayesian import McDropout, MomentPropagation

train_config = TrainConfig(
    batch_size=2,
    epochs=1
)

acq_config = Config(
    name="std_mean",
    pseudo=True
)

dp_model = McDropout(model)
mp_m = MomentPropagation(mp_model)


active_learning = ActiveLearning(
    mp_m, 
    np.expand_dims(new_inputs, axis=-1), labels=new_targets, 
    train_config=train_config,
    acq_name="max_entropy"
)

history = active_learning.start(step_size=400)

100%|██████████| 28/28 [03:54<00:00,  8.37s/it]


In [ ]:
# Compare mc dropout, moment propagation (max_entropy, bald)

In [33]:
history

[{'iteration': 0,
  'train_time': 5.691702365875244,
  'query_time': 2.4125893115997314,
  'loss': [9.504735946655273],
  'tf_op_layer_Sigmoid_1_loss': [0.6676622629165649],
  'tf_op_layer_Mul_99_loss': [8.837074279785156],
  'tf_op_layer_Sigmoid_1_accuracy': [1.0],
  'tf_op_layer_Mul_99_accuracy': [0.0]},
 {'iteration': 400,
  'train_time': 5.7012245655059814,
  'query_time': 2.592912197113037,
  'loss': [9.525849342346191],
  'tf_op_layer_Sigmoid_1_loss': [0.6665551066398621],
  'tf_op_layer_Mul_99_loss': [8.859295845031738],
  'tf_op_layer_Sigmoid_1_accuracy': [1.0],
  'tf_op_layer_Mul_99_accuracy': [0.0]},
 {'iteration': 800,
  'train_time': 5.90375828742981,
  'query_time': 2.428044080734253,
  'loss': [9.525849342346191],
  'tf_op_layer_Sigmoid_1_loss': [0.6665550470352173],
  'tf_op_layer_Mul_99_loss': [8.859294891357422],
  'tf_op_layer_Sigmoid_1_accuracy': [1.0],
  'tf_op_layer_Mul_99_accuracy': [0.0]},
 {'iteration': 1200,
  'train_time': 5.8295838832855225,
  'query_time': 2

In [90]:
%autoreload 2

import active_learning
importlib.reload(active_learning)
from active_learning import Metrics

METRICS_PATH = os.path.join(BASE_PATH, "metrics")


metric_writer = Metrics(METRICS_PATH, keys=["iteration", "train_time", "query_time", "loss"])


all_metrics = []
for entry in history:
    collected = metric_writer.collect(entry)
    all_metrics.append(collected)

In [91]:
all_metrics

[{'iteration': 0,
  'train_time': 5.691702365875244,
  'query_time': 2.4125893115997314,
  'loss': [9.504735946655273]},
 {'iteration': 400,
  'train_time': 5.7012245655059814,
  'query_time': 2.592912197113037,
  'loss': [9.525849342346191]},
 {'iteration': 800,
  'train_time': 5.90375828742981,
  'query_time': 2.428044080734253,
  'loss': [9.525849342346191]},
 {'iteration': 1200,
  'train_time': 5.8295838832855225,
  'query_time': 2.4592678546905518,
  'loss': [9.525850296020508]},
 {'iteration': 1600,
  'train_time': 5.596572399139404,
  'query_time': 2.6522085666656494,
  'loss': [9.525850296020508]},
 {'iteration': 2000,
  'train_time': 5.925888538360596,
  'query_time': 2.619699478149414,
  'loss': [9.525850296020508]},
 {'iteration': 2400,
  'train_time': 5.8023810386657715,
  'query_time': 2.646620750427246,
  'loss': [9.525850296020508]},
 {'iteration': 2800,
  'train_time': 6.186936616897583,
  'query_time': 2.557513952255249,
  'loss': [9.525850296020508]},
 {'iteration': 3

In [92]:
%autoreload 2
metric_writer.write("test", all_metrics)

In [93]:
%autoreload 2
read_in_metrics = metric_writer.read("test")

In [94]:
len(read_in_metrics)

28